### Encoder(P2P) --> Diffusion(DDPM) --> Decoder(P2P)

In [1]:
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from datasets import load_dataset
import matplotlib.pyplot as plt
import wandb
from diffusers import DDPMScheduler
import numpy as np
from tqdm import tqdm
from torchinfo import summary
import torch.cuda.amp as amp


/home/anshul/anaconda3/envs/bhavik_mini/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = {
    "project_name": "Pix2Pix_Diffusion_Pipeline",
    "device": "cuda:1" if torch.cuda.is_available() else "cpu",
    "batch_size": 512,
    "timesteps": 1000,
    "embedding_dim": 512,
    "time_emb_dim": 256,
    "learning_rate": 2e-4,
    "num_epochs": 50,
    "save_checkpoint_interval": 5,
    "diffusion_loss_weight": 0.65,
    "latent_loss_weight": 0.1,
    "sample_interval": 10,
    "val_interval": 10,
    "use_mixed_precision": True,
    "logging": {
        "use_wandb": True,
        "sample_dir": "./outputs/pipeline_samples",
        "checkpoint_dir": "./outputs/pipeline_checkpoints",
        "plot_dir": "./outputs/pipeline_plots"
    }
}

for dir_path in [config["logging"]["sample_dir"], config["logging"]["checkpoint_dir"], config["logging"]["plot_dir"]]:
    try:
        os.makedirs(dir_path, exist_ok=True)
        print(f"📁 Created directory: {dir_path}")
    except Exception as e:
        print(f"⚠️ Failed to create directory {dir_path}: {e}")


📁 Created directory: ./outputs/pipeline_samples
📁 Created directory: ./outputs/pipeline_checkpoints
📁 Created directory: ./outputs/pipeline_plots


In [3]:
from diffusers import DDPMScheduler
import math

class DiffusionAutoencoder(nn.Module):
    """Diffusion Autoencoder combining an encoder, U-Net, and decoder for image reconstruction."""
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.device = torch.device(config["device"])
        self.embedding_dim = config["embedding_dim"]
        self.time_emb_dim = config["time_emb_dim"]
        self.timesteps = config["timesteps"]
        
        self.encoder = self.Encoder(in_channels=3, features=64, embedding_dim=self.embedding_dim)
        self.unet = self.ImprovedMLP_UNet(embedding_dim=self.embedding_dim, time_dim=self.time_emb_dim)
        self.decoder = self.Decoder(out_channels=3, features=64, embedding_dim=self.embedding_dim)
        self.scheduler = self.DiffusionScheduler(timesteps=self.timesteps)
        
        self.to(self.device)

    class SinusoidalPositionEmbeddings(nn.Module):
        """Generates sinusoidal position embeddings for timesteps."""
        def __init__(self, dim):
            super().__init__()
            self.dim = dim

        def forward(self, time):
            device = time.device
            half_dim = self.dim // 2
            embeddings = math.log(10000) / (half_dim - 1)
            embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
            embeddings = time[:, None] * embeddings[None, :]
            embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
            if self.dim % 2 == 1:
                embeddings = F.pad(embeddings, (0, 1, 0, 0))
            return embeddings

    class Encoder(nn.Module):
        """Encodes input images into a latent representation."""
        def __init__(self, in_channels, features, embedding_dim):
            super().__init__()
            self.initial = nn.Sequential(
                nn.Conv2d(in_channels, features, kernel_size=4, stride=2, padding=1, bias=False),
                nn.LeakyReLU(0.2, inplace=True)
            )
            self.down1 = self._block(features, features * 2)
            self.down2 = self._block(features * 2, features * 4)
            self.down3 = self._block(features * 4, features * 8)
            self.down4 = self._block(features * 8, features * 8)
            self.final = nn.Sequential(
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
                nn.Linear(features * 8, embedding_dim)
            )

        def _block(self, in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )

        def forward(self, x):
            d1 = self.initial(x)
            d2 = self.down1(d1)
            d3 = self.down2(d2)
            d4 = self.down3(d3)
            d5 = self.down4(d4)
            embedding = self.final(d5)
            return embedding, [d1, d2, d3, d4, d5]

    class ResidualBlock(nn.Module):
        """Residual block for U-Net with timestep embeddings."""
        def __init__(self, in_channels, out_channels, time_emb_dim, dropout=0.1):
            super().__init__()
            self.time_mlp = nn.Sequential(
                nn.Linear(time_emb_dim, out_channels),
                nn.GELU()
            )
            self.block1 = nn.Sequential(
                nn.Linear(in_channels, out_channels),
                nn.GELU(),
                nn.Dropout(dropout)
            )
            self.block2 = nn.Sequential(
                nn.Linear(out_channels, out_channels),
                nn.GELU(),
                nn.Dropout(dropout)
            )
            self.residual_conv = nn.Linear(in_channels, out_channels) if in_channels != out_channels else nn.Identity()
            self.layer_norm = nn.LayerNorm(out_channels)

        def forward(self, x, t):
            h = self.block1(x)
            time_emb = self.time_mlp(t)
            h = h + time_emb
            h = self.block2(h)
            return self.layer_norm(h + self.residual_conv(x))

    class ImprovedMLP_UNet(nn.Module):
        """U-Net model for denoising latent representations with timestep conditioning."""
        def __init__(self, embedding_dim, hidden_dim=1024, time_dim=256, dropout=0.1):
            super().__init__()
            self.time_mlp = nn.Sequential(
                DiffusionAutoencoder.SinusoidalPositionEmbeddings(time_dim),
                nn.Linear(time_dim, time_dim * 2),
                nn.GELU(),
                nn.Linear(time_dim * 2, time_dim),
            )
            self.down1 = DiffusionAutoencoder.ResidualBlock(embedding_dim, hidden_dim, time_dim, dropout)
            self.down2 = DiffusionAutoencoder.ResidualBlock(hidden_dim, hidden_dim, time_dim, dropout)
            self.mid = DiffusionAutoencoder.ResidualBlock(hidden_dim, hidden_dim, time_dim, dropout)
            self.up1 = DiffusionAutoencoder.ResidualBlock(hidden_dim * 2, hidden_dim, time_dim, dropout)
            self.up2 = DiffusionAutoencoder.ResidualBlock(hidden_dim * 2, embedding_dim, time_dim, dropout)
            self.final = nn.Sequential(
                nn.Linear(embedding_dim, embedding_dim),
                nn.Tanh()
            )

        def forward(self, x, t):
            time_emb = self.time_mlp(t)
            down1 = self.down1(x, time_emb)
            down2 = self.down2(down1, time_emb)
            mid = self.mid(down2, time_emb)
            up1 = self.up1(torch.cat([mid, down2], dim=1), time_emb)
            up2 = self.up2(torch.cat([up1, down1], dim=1), time_emb)
            return self.final(up2)

    class Decoder(nn.Module):
        """Decodes latent representations back to images."""
        def __init__(self, out_channels, features, embedding_dim):
            super().__init__()
            self.project = nn.Sequential(
                nn.Linear(embedding_dim, 512 * 2 * 2),
                nn.ReLU(inplace=True)
            )
            self.up1 = self._block(512, 512)
            self.up2 = self._block(512 + 512, 256)
            self.up3 = self._block(256 + 256, 128)
            self.up4 = self._block(128 + 128, 64)
            self.final = nn.Sequential(
                nn.ConvTranspose2d(64 + 64, out_channels, kernel_size=4, stride=2, padding=1),
                nn.Tanh()
            )

        def _block(self, in_channels, out_channels):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )

        def forward(self, x, encoder_features=None):
            x = self.project(x)
            x = x.view(-1, 512, 2, 2)
            x1 = self.up1(x)
            if encoder_features:
                x2 = self.up2(torch.cat([x1, encoder_features[3]], dim=1))
                x3 = self.up3(torch.cat([x2, encoder_features[2]], dim=1))
                x4 = self.up4(torch.cat([x3, encoder_features[1]], dim=1))
                output = self.final(torch.cat([x4, encoder_features[0]], dim=1))
            else:
                output = self.final(x1)
            return output

    class DiffusionScheduler:
        """Manages noise scheduling for diffusion process using DDPMScheduler."""
        def __init__(self, timesteps):
            self.scheduler = DDPMScheduler(num_train_timesteps=timesteps)

        def add_noise(self, x, t):
            noise = torch.randn_like(x)
            noisy_x = self.scheduler.add_noise(x, noise, t)
            return noisy_x, noise

    def forward(self, img, t):
        """Forward pass through encoder, U-Net, and decoder."""
        latent, encoder_features = self.encoder(img)
        noisy_latent, noise = self.scheduler.add_noise(latent, t)
        denoised_latent = self.unet(noisy_latent, t)
        reconstructed_img = self.decoder(denoised_latent, encoder_features)
        return reconstructed_img, noise, latent, denoised_latent



# purifier = DiffusionAutoencoder(config).to(config['device'])

In [4]:

def sample_images(model, loader, epoch, config, prefix="train", num_samples=4):
    """Generate and save sample images, logging to WandB."""
    model.eval()
    sample_dir = config["logging"]["sample_dir"]
    wandb_images = []
    with torch.no_grad():
        for batch in loader:
            images = batch['image'][:num_samples].to(model.device)
            t = torch.randint(0, model.timesteps, (num_samples,), device=model.device).long()
            output, _, _, _ = model(images, t)
            output = output.cpu().numpy().transpose(0, 2, 3, 1)
            images = images.cpu().numpy().transpose(0, 2, 3, 1)

            output = (output * 0.5 + 0.5).clip(0, 1)
            images = (images * 0.5 + 0.5).clip(0, 1)

            for i in range(num_samples):
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
                ax1.imshow(images[i])
                ax1.set_title("Input")
                ax1.axis("off")
                ax2.imshow(output[i])
                ax2.set_title("Reconstructed")
                ax2.axis("off")

                sample_path = os.path.join(sample_dir, f"{prefix}_epoch_{epoch}_sample_{i}.png")
                plt.savefig(sample_path, bbox_inches="tight")
                plt.close(fig)
                print(f"📸 Saved {prefix} sample: {sample_path}")

                # if config["logging"]["use_wandb"]:
                #     wandb_images.append(wandb.Image(sample_path, caption=f"{prefix.capitalize()} Epoch {epoch} Sample {i}"))

            # if config["logging"]["use_wandb"] and wandb_images:
            #     wandb.log({f"{prefix}_samples": wandb_images}, commit=False)
            break


In [10]:

def validate_model(model, val_loader, config, epoch):
    """Evaluate model on validation set and return loss metrics."""
    model.eval()
    total_loss = 0
    total_recon_loss = 0
    total_diffusion_loss = 0
    total_latent_loss = 0
    num_batches = 0

    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(model.device)
            t = torch.randint(0, model.timesteps, (images.size(0),), device=model.device).long()
            output, noise, latent, denoised_latent = model(images, t)

            recon_loss = F.mse_loss(output, batch['target'].to(model.device))
            diffusion_loss = F.mse_loss(denoised_latent, latent)
            latent_loss = F.mse_loss(denoised_latent, latent.detach())
            loss = recon_loss + config["diffusion_loss_weight"] * diffusion_loss + config["latent_loss_weight"] * latent_loss

            total_loss += loss.item()
            total_recon_loss += recon_loss.item()
            total_diffusion_loss += diffusion_loss.item()
            total_latent_loss += latent_loss.item()
            num_batches += 1

    avg_loss = total_loss / num_batches
    avg_recon_loss = total_recon_loss / num_batches
    avg_diffusion_loss = total_diffusion_loss / num_batches
    avg_latent_loss = total_latent_loss / num_batches

    metrics = {
        "val_loss": avg_loss,
        "val_recon_loss": avg_recon_loss,
        "val_diffusion_loss": avg_diffusion_loss,
        "val_latent_loss": avg_latent_loss
    }
    return metrics



In [11]:
def train_model(model, train_loader, val_loader, config, optimizer=None):
    """Train the diffusion autoencoder with mixed precision and validation."""
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

    scaler = amp.GradScaler(enabled=config["use_mixed_precision"])
    
    # if config["logging"]["use_wandb"]:
        # try:
        #     wandb.init(project=config["project_name"], config=config)
        #     print("✅ WandB initialized")
        # except Exception as e:
        #     print(f"⚠️ WandB initialization failed: {e}")
        #     config["logging"]["use_wandb"] = False

    for epoch in tqdm(range(config["num_epochs"]), desc="Epochs", colour="green"):
        model.train()
        total_loss = 0
        total_recon_loss = 0
        total_diffusion_loss = 0
        total_latent_loss = 0
        num_batches = 0

        for batch in tqdm(train_loader, desc="Batches", leave=False):
            images = batch['image'].to(model.device)
            t = torch.randint(0, model.timesteps, (images.size(0),), device=model.device).long()

            optimizer.zero_grad()
            with amp.autocast(enabled=config["use_mixed_precision"]):
                output, noise, latent, denoised_latent = model(images, t)
                recon_loss = F.mse_loss(output, batch['target'].to(model.device))
                diffusion_loss = F.mse_loss(denoised_latent, latent)
                latent_loss = F.mse_loss(denoised_latent, latent.detach())
                loss = recon_loss + config["diffusion_loss_weight"] * diffusion_loss + config["latent_loss_weight"] * latent_loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            total_recon_loss += recon_loss.item()
            total_diffusion_loss += diffusion_loss.item()
            total_latent_loss += latent_loss.item()
            num_batches += 1

            # if config["logging"]["use_wandb"]:
            #     wandb.log({
            #         "train_loss": loss.item(),
            #         "train_recon_loss": recon_loss.item(),
            #         "train_diffusion_loss": diffusion_loss.item(),
            #         "train_latent_loss": latent_loss.item()
            #     }, commit=False)

        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch + 1}/{config['num_epochs']}, "
              f"Train Loss: {avg_loss:.4f}, "
              f"Recon: {total_recon_loss / num_batches:.4f}, "
              f"Diffusion: {total_diffusion_loss / num_batches:.4f}, "
              f"Latent: {total_latent_loss / num_batches:.4f}")

        if (epoch + 1) % config["val_interval"] == 0:
            val_metrics = validate_model(model, val_loader, config, epoch + 1)
            print(f"Validation - Loss: {val_metrics['val_loss']:.4f}, "
                  f"Recon: {val_metrics['val_recon_loss']:.4f}, "
                  f"Diffusion: {val_metrics['val_diffusion_loss']:.4f}, "
                  f"Latent: {val_metrics['val_latent_loss']:.4f}")
            # if config["logging"]["use_wandb"]:
            #     wandb.log(val_metrics, commit=False)

        if (epoch + 1) % config["sample_interval"] == 0:
            sample_images(model, train_loader, epoch + 1, config, prefix="train")
            sample_images(model, val_loader, epoch + 1, config, prefix="val")

        if (epoch + 1) % config["save_checkpoint_interval"] == 0:
            checkpoint_path = os.path.join(config["logging"]["checkpoint_dir"], f"model_epoch_{epoch + 1}.pth")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"💾 Saved checkpoint: {checkpoint_path}")

        # if config["logging"]["use_wandb"]:
        #     wandb.log({"epoch": epoch + 1, "avg_train_loss": avg_loss}, commit=True)

    final_checkpoint_path = os.path.join(config["logging"]["checkpoint_dir"], "model_final.pth")
    torch.save(model.state_dict(), final_checkpoint_path)
    print(f"💾 Saved final checkpoint: {final_checkpoint_path}")
    print("✅ Training completed!")


In [22]:
class ImagenetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        if self.transform:
            image = self.transform(image)
        noised_image = image + torch.randn_like(image) * torch.rand(1).item()
        return {'image': noised_image, 'target': image}

transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.4802, 0.4481, 0.3975), (0.2302, 0.2265, 0.2262)),
])

dataset = load_dataset("zh-plus/tiny-imagenet", split="train")
dataset = ImagenetDataset(dataset, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=4)
# from torch.utils.data import Dataset
# class ImageNetDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.image_paths = [os.path.join(root, file) for root, _, files in os.walk(root_dir) for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         image = Image.open(img_path).convert("RGB")
#         if self.transform:
#             image = self.transform(image)

#         noised_image = image+torch.randn_like(image)
#         return {'image':noised_image, 'target':image}


# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4802, 0.4481, 0.3975), (0.2302, 0.2265, 0.2262)),
# ])

# imagenet_dir = "./val"
# dataset = ImageNetDataset(root_dir=imagenet_dir, transform=transform)
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=4)
# val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=4)


In [23]:
model = DiffusionAutoencoder(config)

In [24]:
from PIL import Image
train_model(model, train_loader, val_loader, config)

/tmp/ipykernel_1340133/2310151332.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=config["use_mixed_precision"])
Epochs:   0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_1340133/2310151332.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=config["use_mixed_precision"]):
Epochs:   2%|▏         | 1/50 [00:09<07:59,  9.78s/it]

Epoch 1/50, Train Loss: 0.4991, Recon: 0.4912, Diffusion: 0.0104, Latent: 0.0104


Epochs:   4%|▍         | 2/50 [00:18<07:07,  8.90s/it]

Epoch 2/50, Train Loss: 0.3342, Recon: 0.3339, Diffusion: 0.0004, Latent: 0.0004


Epochs:   6%|▌         | 3/50 [00:26<06:40,  8.53s/it]

...

Epoch 49/50, Train Loss: 0.2788, Recon: 0.2788, Diffusion: 0.0000, Latent: 0.0000


Epoch 50/50, Train Loss: 0.2787, Recon: 0.2787, Diffusion: 0.0000, Latent: 0.0000
Validation - Loss: 0.2790, Recon: 0.2790, Diffusion: 0.0000, Latent: 0.0000
📸 Saved train sample: ./outputs/pipeline_samples/train_epoch_50_sample_0.png
📸 Saved train sample: ./outputs/pipeline_samples/train_epoch_50_sample_1.png
📸 Saved train sample: ./outputs/pipeline_samples/train_epoch_50_sample_2.png
📸 Saved train sample: ./outputs/pipeline_samples/train_epoch_50_sample_3.png
📸 Saved val sample: ./outputs/pipeline_samples/val_epoch_50_sample_0.png
📸 Saved val sample: ./outputs/pipeline_samples/val_epoch_50_sample_1.png
📸 Saved val sample: ./outputs/pipeline_samples/val_epoch_50_sample_2.png
📸 Saved val sample: ./outputs/pipeline_samples/val_epoch_50_sample_3.png


Epochs: 100%|██████████| 50/50 [07:27<00:00,  8.94s/it]

💾 Saved checkpoint: ./outputs/pipeline_checkpoints/model_epoch_50.pth


💾 Saved final checkpoint: ./outputs/pipeline_checkpoints/model_final.pth
✅ Training completed!
